# Gridworld

### Description

In this seminar we implement an example of the agent-environment interface used in reinforcement learning, called "gridworld".

The world consists of an $n \times m$ grid of squares, indexed by $(i,j)$ with $i = 0, 1, \dots, n-1$ and $j = 0, 1, \dots, m-1$.

The state of the environment consists of the player being in one of the squares.

The possible actions are steps in the directions "UP", "RIGHT", "DOWN", "LEFT".

The rewards and states after each action can depend on multiple factors:
- Some squares can give positive/negative rewards
- Squares might be "blocked" and not possible to step on
- Stepping off the board is impossible
- Invalid moves might give a negative reward as "punishment"
- There might be "portals" that take the player to a distant square, regardless of their action
- There might be deterministic or random effects that change the outcome of actions, e.g. "wind" or "ice".

Variants of this gridworld can be used to illustrate a wide range of concepts and algorithms in reinforcement learning.
For instance, in [Sutton & Barto](http://incompleteideas.net/book/the-book-2nd.html) see:
- Example 3.5, 3.8
- Figure 4.1
- Example 6.5, 6.6
- Figure 7.4
- Example 8.1, 8.3


You can either implement this class in a separate module (recommended) or inside a Jupyter cell, see `reuseCode.ipynb` for details.

### Implementation

Below is a suggested skeleton of a `GridWorld` class, feel free to modify or rename everything.
If you want to implement this class in a separate module, create a new file `gridworld.py`, copy the code there, and delete this code cell.

In [109]:
import random
import numpy as np

In [64]:
RIGHT = np.array((1, 0))
LEFT = np.array((-1, 0))
UP = np.array((0, 1))
DOWN = np.array((0, -1))
actions = (RIGHT, LEFT, UP, DOWN)

In [70]:
class GridWorld:

    def __init__(self, height: int, width: int):
        # Store the height and width as attributes
        if height < 5 or width < 5:
            raise Exception('Dimensions of gridworld too small')
        
        self.height = height
        self.width = width
        
        # Initialize the player position
        self.pos = np.array((0, 0))
        
    # A method to perform an action:
    def step(self, action):
        if not any(np.array_equal(action, direction) for direction in actions):
            raise Exception('Wrong input')
        
        if np.array_equal(self.pos, np.array((1, 4))):
            reward = 10
            self.pos = np.array((1, 0))
        elif np.array_equal(self.pos, np.array((3, 4))):
            reward = 5
            self.pos = np.array((3, 2))
        else:
            potential_pos = self.pos + action
            valid_pos_bool = 0 <= potential_pos[0] < self.width and 0 <= potential_pos[1] < self.height
            if valid_pos_bool:
                self.pos = potential_pos
                reward = 0
            else:
                reward = -1
        
        return self.pos, reward

    # A method to reset the gridworld:
    def reset(self):
        self.pos = np.array((0, 0))
    
    # A method to output the world:
    def drawWorld(self):

        state_line = ['_']*self.width
        state_line[self.pos[0]] = 'X'
        state_line = ' '.join(state_line)
        empty_line = ' '.join(['_']*self.width)

        grid = [empty_line]*self.height
        grid[-self.pos[1] + self.height - 1] = state_line
        grid = '\n'.join(grid)
        print(grid)

    def __str__(self):
        state_line = ['_']*self.width
        state_line[self.pos[0]] = 'X'
        state_line = ' '.join(state_line)
        empty_line = ' '.join(['_']*self.width)

        grid = [empty_line]*self.height
        grid[-self.pos[1] + self.height - 1] = state_line
        grid = '\n'.join(grid)
        
        return grid
    
    # (!) More difficult:
    # A method to interactively "play" in the gridworld:
    # def play(??):
    #     ...
    

    # Any other method that might be useful to the user
    # ...

    # Any "helper" methods you use internally
    # ...
    

Once you have implemented the basic methods above, you should be able to walk around in an empty gridworld! 🎉

To make things more interesting, implement for example:
- Positive rewards for reaching certain squares
- Negative rewards for "illegal" moves
- Blocked squares that cannot be entered
- Teleporting squares that move the player to another spot. This is useful to avoid optimal "back-and-forth" policies.
- A `.previewMove()` method to simulate given actions from a given start square.
- A random effect (ice, wind, ...) that changes the effect of some actions.


### Testing

Below you can test individual aspects of your gridworld class with short code cells.

You can re-run cells or run them out of order, but it is recommended that the notebook still works if you `Run All` in a fresh jupyter session.

If you implemented an interactive `.play()` method, you might not be able to test it from inside a jupyter notebook.

In [71]:
# Import the GridWorld class from `gridworld.py`
# from gridworld import GridWorld

In [81]:
# Create a new gridworld instance
gw = GridWorld(5, 5)

In [82]:
# Take some arbitrary actions
gw.drawWorld()
print('\n')
gw.step(UP)
gw.drawWorld()
print('\n')
gw.step(LEFT)
gw.drawWorld()
print('\n')
gw.step(RIGHT)
gw.drawWorld()
print('\n')
gw.step(UP)
gw.drawWorld()

_ _ _ _ _
_ _ _ _ _
_ _ _ _ _
_ _ _ _ _
X _ _ _ _


_ _ _ _ _
_ _ _ _ _
_ _ _ _ _
X _ _ _ _
_ _ _ _ _


_ _ _ _ _
_ _ _ _ _
_ _ _ _ _
X _ _ _ _
_ _ _ _ _


_ _ _ _ _
_ _ _ _ _
_ _ _ _ _
_ X _ _ _
_ _ _ _ _


_ _ _ _ _
_ _ _ _ _
_ X _ _ _
_ _ _ _ _
_ _ _ _ _


In [123]:
# Test any other behaviour that you implemented...
T = 100

gw = GridWorld(5, 5)
totalRandom = 0
for t in range(T):
    action = random.randint(0, 3)
    action = actions[action]
    _, reward = gw.step(action)
    totalRandom = totalRandom + reward
    # print(reward)
    # gw.drawWorld()

totalRandom

1

In [124]:
def play_random_walk(T: int):
    gw = GridWorld(5, 5)
    totalRandom = 0
    for _ in range(T):
        action = random.randint(0, 3)
        action = actions[action]
        _, reward = gw.step(action)
        totalRandom = totalRandom + reward
    return totalRandom

In [126]:
N = 10000
returns_lists = [0]*N
for i in range(N):
    G = play_random_walk(T=100)
    returns_lists[i] = G

In [128]:
mean_return = sum(returns_lists) / N
mean_return

-3.651